In [1]:
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [8]:
winter_df = pd.read_csv('Bike share ridership 2023-01.csv',encoding='ISO-8859-1')  # Winter: January
spring_df = pd.read_csv('Bike share ridership 2023-04.csv',encoding='ISO-8859-1')  # Spring: April
summer_df = pd.read_csv('Bike share ridership 2023-07.csv',encoding='ISO-8859-1')  # Summer: July
fall_df = pd.read_csv('Bike share ridership 2023-10.csv',encoding='ISO-8859-1')    # Fall: October

seasons = [winter_df,spring_df,summer_df,fall_df]
df_list = []
counter = 1

In [9]:
for index, season in enumerate(seasons):
    df = season.copy()
    df.fillna(9999, inplace=True)
    df['Start Time'] = pd.to_datetime(df['Start Time'], format='%m/%d/%Y %H:%M', errors='coerce')
    df['End Time'] = pd.to_datetime(df['End Time'], format='%m/%d/%Y %H:%M', errors='coerce')
    df['End Station Id'] = df['End Station Id'].astype('Int64')


    start_station_ids = df['Start Station Id'].unique()
    end_station_ids = df['End Station Id'].unique()

    station_ids = list(set(start_station_ids).union(end_station_ids))

    earliest_time = min(df['Start Time'])
    last_time = max(df['End Time'])

    time_stamps = pd.date_range(earliest_time, last_time, freq='T')

    time_list = [t.strftime('%Y-%m-%d %H:%M') for t in time_stamps]

    #initializing final dataframe
    df_final0 = pd.DataFrame(index=time_list, columns=station_ids)
    df_final0 = df_final0.fillna(0)
    bike_id_list = df['Bike Id'].unique().tolist()
    for i in bike_id_list:
        try:
            print(str(bike_id_list.index(i))+'/'+str(len(bike_id_list)))
            bike_trans = df[df['Bike Id']==i]
            bike_trans = bike_trans[['Bike Id','Start Station Id','End Station Id','Start Time','End Time']].sort_values(by='Start Time',ascending=True).reset_index()
            for index, row in bike_trans.iterrows():
                c_start_stn_id = row['Start Station Id']
                c_end_stn_id = row['End Station Id']
                c_start_time = row['Start Time'].strftime('%Y-%m-%d %H:%M')
                c_end_time = row['End Time'].strftime('%Y-%m-%d %H:%M')
                if index == 0:
                    df_final0.loc[:c_start_time,c_start_stn_id] = df_final0.loc[:c_start_time,c_start_stn_id] + 1
                    df_final0.loc[c_end_time, c_end_stn_id] = df_final0.loc[c_end_time, c_end_stn_id] + 1
                else:
                    p_end_stn_id = bike_trans.iloc[index-1]['End Station Id']
                    p_end_time = bike_trans.iloc[index-1]['End Time'].strftime('%Y-%m-%d %H:%M')
                    if p_end_stn_id == c_start_stn_id:
                        df_final0.loc[p_end_time:c_start_time,c_start_stn_id] = df_final0.loc[p_end_time:c_start_time,c_start_stn_id] + 1
                        df_final0.loc[c_end_time, c_end_stn_id] = df_final0.loc[c_end_time, c_end_stn_id] + 1
                    else:
                        time1 = datetime.strptime(p_end_time,'%Y-%m-%d %H:%M')
                        time2 = datetime.strptime(c_start_time,'%Y-%m-%d %H:%M')
                        avg_time = time1 + (time2 - time1) / 2
                        avg_time_minus_one_min = avg_time - timedelta(minutes=1)
                        avg_time_minus_one_min = avg_time_minus_one_min.strftime('%Y-%m-%d %H:%M')
                        avg_time = avg_time.strftime('%Y-%m-%d %H:%M')
                        df_final0.loc[p_end_time:avg_time_minus_one_min,p_end_stn_id] = df_final0.loc[p_end_time:avg_time_minus_one_min,p_end_stn_id] + 1
                        df_final0.loc[avg_time:c_start_time,c_start_stn_id] = df_final0.loc[avg_time:c_start_time,c_start_stn_id] + 1
                        df_final0.loc[c_end_time, c_end_stn_id] = df_final0.loc[c_end_time, c_end_stn_id] + 1
                if index == len(bike_trans) - 1:
                    c_end_time_str = datetime.strptime(c_end_time,'%Y-%m-%d %H:%M')
                    min_after_end_time = c_end_time_str - timedelta(minutes=1)
                    min_after_end_time = min_after_end_time.strftime('%Y-%m-%d %H:%M')
                    df_final0.loc[min_after_end_time:,c_end_stn_id] = df_final0.loc[min_after_end_time:,c_end_stn_id] + 1
        except:
            pass  
    df_final0 = df_final0.reset_index()
    df_list.append(df_final0)

df_combined = pd.concat(df_list, ignore_index=True)


0/5741
1/5741
2/5741
3/5741
4/5741
5/5741
6/5741
7/5741
8/5741
9/5741
10/5741
11/5741
12/5741
13/5741
14/5741
15/5741
16/5741
17/5741
18/5741
19/5741
20/5741
21/5741
22/5741
23/5741
24/5741
25/5741
26/5741
27/5741
28/5741
29/5741
30/5741
31/5741
32/5741
33/5741
34/5741
35/5741
36/5741
37/5741
38/5741
39/5741
40/5741
41/5741
42/5741
43/5741
44/5741
45/5741
46/5741
47/5741
48/5741
49/5741
50/5741
51/5741
52/5741
53/5741
54/5741
55/5741
56/5741
57/5741
58/5741
59/5741
60/5741
61/5741
62/5741
63/5741
64/5741
65/5741
66/5741
67/5741
68/5741
69/5741
70/5741
71/5741
72/5741
73/5741
74/5741
75/5741
76/5741
77/5741
78/5741
79/5741
80/5741
81/5741
82/5741
83/5741
84/5741
85/5741
86/5741
87/5741
88/5741
89/5741
90/5741
91/5741
92/5741
93/5741
94/5741
95/5741
96/5741
97/5741
98/5741
99/5741
100/5741
101/5741
102/5741
103/5741
104/5741
105/5741
106/5741
107/5741
108/5741
109/5741
110/5741
111/5741
112/5741
113/5741
114/5741
115/5741
116/5741
117/5741
118/5741
119/5741
120/5741
121/5741
122/5741
123

In [15]:
df_combined = pd.read_csv('final_formatted_df.csv')

In [16]:
df_combined['date'] = pd.to_datetime(df_combined['index'], format='%Y-%m-%d %H:%M')
df_combined['week_number'] = df_combined['date'].dt.isocalendar().week
df_combined['date_only'] = df_combined['date'].dt.date
week_nm_keep = [3,16,30,45]
df_filtered = df_combined[df_combined['week_number'].isin(week_nm_keep)]
df_filtered.reset_index(inplace=True)
df_filtered = df_filtered.drop(columns=['level_0','Unnamed: 0'])
df_filtered = df_filtered.fillna(0)
df_filtered.rename(columns={'index': 'timestamp'}, inplace=True)
df_filtered = df_filtered.drop(columns=['date','week number'])
float_columns = df_filtered.select_dtypes(include='float').columns
df_filtered[float_columns] = df_filtered[float_columns].astype(int)
df_melted = pd.melt(df_filtered, id_vars=['timestamp','date_only','week_number'], var_name='station_id', value_name='bike_count')
df_final = df_melted.copy()
seasons = ['winter','spring','summer','fall']
week_numbers = [3,16,30,45]
df_final['season'] = df_final['week_number'].apply(lambda x: seasons[week_numbers.index(x)])
df_final['timestamp_dt'] = pd.to_datetime(df_final['timestamp'], format='%Y-%m-%d %H:%M')
df_final['day_of_week'] = df_final['timestamp_dt'].dt.dayofweek
df_final['Weekend Tag'] = np.where(df_final['day_of_week'].isin([5,6]),1,0)
df_final['hour'] = df_final['timestamp_dt'].dt.hour
df_final['rush hour tag'] = np.where((df_final['hour'].isin([5,6,7,8,9,2,3,4,5,6])) & (df_final['Weekend Tag']== 0), 1,0)
df_final['min'] = df_final['timestamp_dt'].dt.minute
df_final_cleaned = df_final[['station_id','bike_count','season','day_of_week','Weekend Tag','hour','rush hour tag','min']]

In [30]:
df_final_cleaned = pd.read_csv('cleaned_final_for_model.csv')
df_final_cleaned = df_final_cleaned[df_final_cleaned['station_id'] != 9999]
df_final_cleaned.drop(columns=['Unnamed: 0'],inplace=True)
q3 = df_final_cleaned['bike_count'].quantile(0.75)
q1 = df_final_cleaned['bike_count'].quantile(0.25)
iqr = q3 - q1
outlier = q3 + (3.5*iqr)
testdf = df_final_cleaned[df_final_cleaned['bike_count']<= outlier]
group_columns = ['station_id', 'season','day_of_week', 'hour']
df_grouped = testdf.groupby(group_columns).agg({'bike_count': 'mean'}).reset_index()
df_grouped.rename(columns={'bike_count': 'average_bike_count'}, inplace=True)
df_grouped['average_bike_count'] = df_grouped['average_bike_count'].apply(np.ceil).astype(int)
df_grouped.sort_values(by=['station_id','season','day_of_week','hour'])
df_grouped['moving_average_bike_count'] = df_grouped.groupby(['station_id', 'season', 'day_of_week'])['average_bike_count'].transform(lambda x: x.rolling(window=3, min_periods=1).mean())
df_grouped['moving_average_bike_count'] = np.ceil(df_grouped['moving_average_bike_count'])
df_grouped.to_csv('final_data_for_app.csv')
df_grouped

,station_id,season,day_of_week,hour,average_bike_count,moving_average_bike_count
0,7000,fall,0,0,34,34.0
1,7000,fall,0,1,34,34.0
2,7000,fall,0,2,34,34.0
3,7000,fall,0,3,34,34.0
4,7000,fall,0,4,34,34.0
...,...,...,...,...,...,...
506608,7864,winter,6,19,0,0.0
506609,7864,winter,6,20,0,0.0
506610,7864,winter,6,21,0,0.0
506611,7864,winter,6,22,0,0.0


In [4]:
file_path = 'lng_lat.txt'  
with open(file_path, 'r') as file:
    data = file.read()
data_clean = data.replace("{", "").replace("}", "").replace(" ", "").strip()
station_data = [entry.split(":") for entry in data_clean.split(",")]
station_ids = []
latitudes = []
longitudes = []
for entry in station_data:
    station_id = entry[0]
    lat_lng = entry[1].split("_")
    latitude = lat_lng[0]
    longitude = lat_lng[1]
    
    station_ids.append(station_id)
    latitudes.append(latitude)
    longitudes.append(longitude)

df_lng_lat = pd.DataFrame({
    'Station_ID': station_ids,
    'lat': latitudes,
    'lng': longitudes
})
df_lng_lat.to_csv('df_lng_lat.csv')


In [32]:
import re
with open('locations.txt', 'r', encoding='latin-1') as file:
    data = file.read()
li_id_pattern = re.compile(r'<li id="(\d+)">')
green_title_pattern = re.compile(r'<div class="greenTitles marginBtm20">(.*?)</div>')
li_ids = li_id_pattern.findall(data)
green_titles = green_title_pattern.findall(data)
df_locations = pd.DataFrame({
    'ID': li_ids,
    'Title': green_titles
})

df_locations.to_csv('location_names.csv')